[![Build Status](https://travis-ci.org/chrisjsewell/jsonextended.svg?branch=master)](https://travis-ci.org/chrisjsewell/jsonextended)
[![Coverage Status](https://coveralls.io/repos/github/chrisjsewell/jsonextended/badge.svg?branch=master)](https://coveralls.io/github/chrisjsewell/jsonextended?branch=master)

# JSON Extended

A module to extend the python json package functionality: 

-  decoding/encoding between the on-disk JSON structure
   and in-memory nested dictionary structure, including:

   -  treating path structures, with nested directories and multiple .json files, as a single json.

   -  on-disk indexing of the json structure (using the ijson package)

   -  extended data type serialisation (numpy.ndarray, Decimals,
      pint.Quantities,...)

-  viewing and manipulating the nested dictionaries:

   -  enhanced pretty printer
   
   -  Javascript rendered, expandable tree in the Jupyter Notebook
   
   -  filter, merge, flatten, unflatten functions

-  Units schema concept to apply and convert physical units (using the
   pint package)

-  Parser abstract class for dealing with converting other file formats
   to JSON

## Example

### Data Folders JSONisation

In [2]:
from jsonextended import ejson, edict

In [3]:
path = ejson.get_test_path()
ejson.jkeys(path)

['dir1', 'dir2', 'dir3']

In [4]:
jdict1 = ejson.to_dict(path)
edict.pprint(jdict1,depth=2)

dir1: 
  dir1_1: {...}
  file1: {...}
  file2: {...}
dir2: 
  file1: {...}
dir3: 


In [5]:
edict.to_html(jdict1,depth=2)

### Nested Dict Manipulation

In [6]:
jdict2 = ejson.to_dict(path,['dir1','file1'])
edict.pprint(jdict2,depth=1)

initial: {...}
meta: {...}
optimised: {...}
units: {...}


In [7]:
filtered = edict.filter_keys(jdict2,['vol*'],use_wildcards=True)
edict.pprint(filtered)

initial: 
  crystallographic: 
    volume: 924.62752781
  primitive: 
    volume: 462.313764
optimised: 
  crystallographic: 
    volume: 1063.98960509
  primitive: 
    volume: 531.994803


In [8]:
edict.pprint(edict.flatten(filtered))

('initial', 'crystallographic', 'volume'):   924.62752781
('initial', 'primitive', 'volume'):          462.313764
('optimised', 'crystallographic', 'volume'): 1063.98960509
('optimised', 'primitive', 'volume'):        531.994803


### Units Schema

In [9]:
from jsonextended.units import apply_unitschema, split_quantities
withunits = apply_unitschema(filtered,{'volume':'angstrom^3'})
edict.pprint(withunits)

initial: 
  crystallographic: 
    volume: 924.62752781 Å ** 3
  primitive: 
    volume: 462.313764 Å ** 3
optimised: 
  crystallographic: 
    volume: 1063.98960509 Å ** 3
  primitive: 
    volume: 531.994803 Å ** 3


In [10]:
newunits = apply_unitschema(withunits,{'volume':'nm^3'})
edict.pprint(newunits)

initial: 
  crystallographic: 
    volume: 0.92462752781 nm ** 3
  primitive: 
    volume: 0.462313764 nm ** 3
optimised: 
  crystallographic: 
    volume: 1.06398960509 nm ** 3
  primitive: 
    volume: 0.531994803 nm ** 3


In [11]:
edict.pprint(split_quantities(newunits),depth=4)

initial: 
  crystallographic: 
    volume: 
      magnitude: 0.92462752781
      units:     nanometer ** 3
  primitive: 
    volume: 
      magnitude: 0.462313764
      units:     nanometer ** 3
optimised: 
  crystallographic: 
    volume: 
      magnitude: 1.06398960509
      units:     nanometer ** 3
  primitive: 
    volume: 
      magnitude: 0.531994803
      units:     nanometer ** 3
